This notebook trains six models of each of the four architectures.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from supportFunctions import createModel
import time

In [2]:
# This is a custom callback that allows timing the training process
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [3]:
# RETRIEVE images from directories

image_size = (128, 128)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-training-and-validation",
    validation_split=0.25,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode = "grayscale"
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-training-and-validation",
    validation_split=0.25,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode="grayscale"
)

Found 17600 files belonging to 2 classes.
Using 13200 files for training.
Found 17600 files belonging to 2 classes.
Using 4400 files for validation.


In [4]:
modelNames = ("tuned2", "custom3", "tuned3", "tuned5")
numberOfModelsToProduce = 6
epochs=1000

METRICS = [
]

# set up early stopping, runs for at least 15 epochs, no minimum improvement required
early_stopping= tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    min_delta=0.00,
                    patience=15,
                    verbose=1,
                    mode='min',
                    baseline=None,
                    restore_best_weights=True
                )
            
time_callback = TimeHistory()

In [5]:
# train each of the 24 models and save them, their histories, and their timings
for i in np.arange(1,numberOfModelsToProduce+1):
    for modelName in modelNames:
        model = createModel(modelName = modelName)
        model.compile(
            optimizer=keras.optimizers.Adam(1e-3),
            loss="binary_crossentropy",
            metrics=METRICS,
        )
        
        
        history = model.fit(
            train_ds, epochs=epochs, callbacks=[early_stopping, time_callback], validation_data=val_ds
        )
        
        
        np.save("history_%s-%s.npy" % (modelName, i),history.history)
        
        np.save("times_%s-%s.npy" % (modelName, i), time_callback.times)
        
        model.save("modelDirectory-%s-%s" % (modelName, i))

Epoch 1/1000
413/413 [==============================] - 66s 133ms/step - loss: 0.2522 - val_loss: 0.1745
Epoch 2/1000
413/413 [==============================] - 47s 113ms/step - loss: 0.1603 - val_loss: 0.1432
Epoch 3/1000
413/413 [==============================] - 47s 114ms/step - loss: 0.1408 - val_loss: 0.1600
Epoch 4/1000
413/413 [==============================] - 47s 114ms/step - loss: 0.1193 - val_loss: 0.1726
Epoch 5/1000
413/413 [==============================] - 47s 114ms/step - loss: 0.1022 - val_loss: 0.0984
Epoch 6/1000
413/413 [==============================] - 48s 115ms/step - loss: 0.0821 - val_loss: 0.0803
Epoch 7/1000
413/413 [==============================] - 48s 115ms/step - loss: 0.0720 - val_loss: 0.0732
Epoch 8/1000
413/413 [==============================] - 48s 115ms/step - loss: 0.0658 - val_loss: 0.0886
Epoch 9/1000
413/413 [==============================] - 47s 113ms/step - loss: 0.0610 - val_loss: 0.0779
Epoch 10/1000
413/413 [==============================] 